# setup

In [5]:
import sys
sys.path.append("..")

from forwardKinematics import robot_config
import numpy as np
import time

pi = np.pi
myRobot = robot_config()

# check forward kinematics speed and behavior

In [10]:
q = np.array([0.5, 0.25, 0., 0., 0., 0., 0.])
print("Joint angles/positions {}".format(q))


s = time.time()
for i in range(1000):
	positions = myRobot.forwardKinPos(q)
dt = time.time()-s
print(dt/1000)
print('Forward kinematic positions: {}'.format(positions))

s = time.time()
for i in range(100):
	orientation = myRobot.forwardKinOrientation(q)
dt = time.time()-s
print(dt/100)
print('Forward kinematic orientations: {}'.format(orientation))

Joint angles/positions [0.5  0.25 0.   0.   0.   0.   0.  ]
0.0013133034706115723
Forward kinematic positions: [[ 0.00000000e+00  5.00000000e-01  3.06161700e-17]
 [-2.50000000e-01  5.00000000e-01  6.12323400e-17]
 [-2.05000000e+00  5.00000000e-01  2.81668764e-16]
 [-2.16300000e+00  5.00000000e-01  2.95507273e-16]
 [-2.24300000e+00  5.00000000e-01  3.10203034e-16]
 [-2.32300000e+00  5.00000000e-01  3.24898796e-16]
 [-2.49800000e+00  5.00000000e-01  2.30000000e-02]]
0.0015666913986206055
Forward kinematic orientations: [[ 0.00000000e+00 -7.07106781e-01 -7.07106781e-01  4.32978028e-17]
 [ 5.00000000e-01 -5.00000000e-01 -5.00000000e-01  5.00000000e-01]
 [ 5.00000000e-01 -5.00000000e-01 -5.00000000e-01  5.00000000e-01]
 [ 5.00000000e-01 -5.00000000e-01  5.00000000e-01  5.00000000e-01]
 [ 1.66533454e-16 -2.22044605e-16  7.07106781e-01  7.07106781e-01]
 [ 5.00000000e-01 -5.00000000e-01  5.00000000e-01  5.00000000e-01]
 [ 0.00000000e+00 -7.07106781e-01 -4.32978028e-17  7.07106781e-01]]


In [11]:

# calculate position of the end-effector
#xyz = myRobot.Tx('EE', q)
#print('XYZ forward kin: {}'.format(xyz))
#print(xyz.shape)
#print(type(xyz))
# calculate the Jacobian for the end effector
s = time.time()
for i in range(1000):
	JEE = myRobot.J('j4', q)
dt = time.time()-s
print(dt/1000)
print("analytic jacobian: \n{}".format(JEE))

def calculateJacobian(q, num_joints = 7, dx = 1e-5):
	jacobian = np.zeros((6,num_joints))
	for i in range(num_joints):
		q_plus = q.copy()
		q_plus[i] += dx
		q_minus = q.copy()
		q_minus[i] -= dx
		x_plus = myRobot.Tx('EE', q_plus)
		x_minus = myRobot.Tx('EE', q_minus)
		jacobian[:3, i] = (x_plus - x_minus) / (2*dx)

		qEE_plus = myRobot.Te('EE', q_plus)
		qEE_minus = myRobot.Te('EE', q_minus)
		jacobian[3:, i] = (qEE_plus - qEE_minus) / (2*dx)

	return jacobian

s = time.time()
for i in range(1000):
	JEE_numerical = calculateJacobian(q)
dt = time.time()-s
print(dt/1000)
print("numeric jacobian: \n{}".format(JEE_numerical))



#J_orientation = myRobot._calc_T('j1', lambdify = True)
#print(J_orientation(*tuple(q)))

s = time.time()
for i in range(1000):
	orientation = myRobot.forwardKinOrientation(q)
dt = time.time()-s
print(dt/1000)
print('Forward kinematic orientations: {}'.format(orientation))

s = time.time()
for i in range(1000):
	test = myRobot.forwardKinHomogenous(q)
dt = time.time()-s
print(dt/1000)

7.526087760925292e-05
analytic jacobian: 
[[ 0.0000000e+00 -1.0000000e+00  0.0000000e+00 -0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 1.0000000e+00  6.1232340e-17  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 6.1232340e-17  1.2246468e-16  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -1.0000000e+00 -1.0000000e+00 -6.1232340e-17
   0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 1.0000000e+00  6.1232340e-17  6.1232340e-17 -1.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 6.1232340e-17  1.2246468e-16  1.2246468e-16 -1.2246468e-16
   0.0000000e+00  0.0000000e+00  0.0000000e+00]]
0.02265511178970337
numeric jacobian: 
[[ 0.00000000e+00 -1.00000000e+00  0.00000000e+00 -2.30000000e-02
   0.00000000e+00 -2.30000000e-02 -1.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00  2.30000000e-02  0.00000000e+00
  -2.55000000e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.000